In [6]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
from torchvision import transforms, datasets
import torchvision

In [7]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
from torchvision import transforms, utils
# ----------------------------
# Sound Dataset
# ----------------------------
class SongLabelSet(Dataset):
  def __init__(self, df):
    self.df = df

  # ----------------------------
  # Number of items in dataset
  # ----------------------------
  def __len__(self):
    return len(self.df)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    values = [
        self.df["acousticness"][idx] *100,
        self.df["danceability"][idx]*100,
        self.df["energy"][idx]*100,
        self.df["instrumentalness"][idx]*100,
        self.df["liveness"][idx]*10,
        self.df["speechiness"][idx]*100,
        self.df["valence"][idx]*100,
    ]
    values = torch.tensor(values, dtype=torch.float32)
    label = self.df["cluster_id"][idx]
    return values, label


In [8]:
songDataset = pd.read_csv("./allSongsLabelwClusters.csv", low_memory=False)

In [9]:
songData = SongLabelSet(songDataset)

In [10]:
songData[1]

(tensor([73.9485, 69.3053, 59.0173, 33.9241,  1.6437, 19.8380, 65.0165]), 18.0)

In [11]:
num_items = len(songDataset)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(songData, [num_train, num_val])
num_items
len(val_ds)

10444

In [12]:
input_transform = transforms.Compose([    transforms.Lambda(lambda x: x * 100)])

label_transform = transforms.Compose([    transforms.ToTensor()])

transform = transforms.Compose([    input_transform,    label_transform])

In [13]:
batch_size = 32
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False)

In [14]:
for i in train_dl:
    print(i)
    break

[tensor([[11.0933, 72.7885, 67.8329, 73.3009,  1.0595, 12.1789, 46.4053],
        [77.9101, 43.2217,  5.5453, 95.9648,  0.6803,  5.0906, 14.5733],
        [ 0.2106, 54.7741, 63.8926, 96.4582,  0.8998,  4.9114,  3.3780],
        [98.7361, 76.2851, 42.4639, 28.4628,  1.4745, 32.3302, 77.8634],
        [96.4298, 24.9454, 17.8889, 83.0754,  0.7594,  2.4143,  6.1182],
        [97.0948, 31.6164,  8.1904, 83.6127,  3.1820, 23.1909, 49.4755],
        [77.9206, 35.8567,  6.5742, 16.3842,  3.5989, 56.6895, 45.1630],
        [66.9586, 75.5273, 75.1581, 42.9142,  1.1659, 18.5195, 33.3195],
        [ 1.2010, 83.4820, 47.5222, 99.6540,  0.3887,  8.9668, 12.6098],
        [68.6842, 77.4376, 41.7275, 71.4198,  0.8003,  9.3376, 35.7559],
        [ 1.0739, 61.8766, 68.3391, 96.4247,  0.8639,  6.6797,  6.9204],
        [90.3216, 48.5460, 36.3822, 65.8810,  1.0165,  7.3951, 38.7369],
        [97.8087, 40.2563,  5.9498, 91.4846,  1.6575, 13.0396, 22.6820],
        [18.6222, 84.8196, 49.0981, 94.6739,  0.55

In [16]:
import torch
import torch.nn as nn

class LabelToClusterModel(nn.Module):
    def __init__(self):
        super(LabelToClusterModel, self).__init__()
        self.fc1 = nn.Linear(7, 512)
        self.relu1 = nn.ReLU()
#         self.relu1 = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
#         self.relu2 = nn.LeakyReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()
#         self.relu3 = nn.LeakyReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(128, 500)
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.zeros_(self.fc3.bias)
        nn.init.xavier_uniform_(self.fc4.weight)
        nn.init.zeros_(self.fc4.bias)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
#         x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
#         x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
#         x = self.dropout3(x)
        x = self.fc4(x)
#         x = self.softmax(x)
#         self.sigmoid(x)
        return x

In [17]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [18]:
device

device(type='cpu')

In [19]:
model = LabelToClusterModel()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
print(next(model.parameters()).device)

cpu


In [20]:
def trainLabel(model, optimizer, criterion, trainloader, testloader, num_epochs):
    model.train()
    stats = [[],[],[],[]]
#     print(device)
    for epoch in range(num_epochs):
        train_loss = 0.0
        train_acc = 0.0
        val_loss = 0.0
        val_acc = 0.0
        for data in trainloader:
            
            inputs, labels = data
#             inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.type(torch.LongTensor)
#             print(inputs)
#             print(labels)
            optimizer.zero_grad()

            outputs = model(inputs)
#             print(outputs)
            predicted_class = torch.argmax(outputs, dim=1)
#             print(predicted_class)
#             break
            loss = criterion(outputs, labels)
#             print(loss)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_acc += (predicted == labels).sum().item()
            

 # Validation loop
        with torch.no_grad():
            for inputs, labels in testloader:
#                 inputs, labels = inputs.to(device), labels.to(device)
                labels = labels.type(torch.LongTensor)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_acc += (predicted == labels).sum().item()
        
        # Print epoch statistics
        train_loss /= len(trainloader.dataset)
        stats[0].append(train_loss)
        train_acc /= len(trainloader.dataset)
        stats[1].append(train_acc)
        val_loss /= len(testloader.dataset)
        stats[2].append(val_loss)
        val_acc /= len(testloader.dataset)
        stats[3].append(val_acc)
        print(f'Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')

    print("Finished training")
    return stats

In [166]:
stats1 = trainLabel(model, optimizer, criterion, train_dl, val_dl, 20)

Epoch 1: Train Loss=0.1744, Train Acc=0.0166, Val Loss=0.1519, Val Acc=0.0341
Epoch 2: Train Loss=0.1369, Train Acc=0.0550, Val Loss=0.1279, Val Acc=0.0742
Epoch 3: Train Loss=0.1195, Train Acc=0.0876, Val Loss=0.1147, Val Acc=0.1041
Epoch 4: Train Loss=0.1116, Train Acc=0.1100, Val Loss=0.1101, Val Acc=0.1153
Epoch 5: Train Loss=0.1073, Train Acc=0.1285, Val Loss=0.1066, Val Acc=0.1267
Epoch 6: Train Loss=0.1052, Train Acc=0.1352, Val Loss=0.1048, Val Acc=0.1393
Epoch 7: Train Loss=0.1026, Train Acc=0.1464, Val Loss=0.1010, Val Acc=0.1543
Epoch 8: Train Loss=0.1004, Train Acc=0.1591, Val Loss=0.1028, Val Acc=0.1523
Epoch 9: Train Loss=0.0987, Train Acc=0.1643, Val Loss=0.0991, Val Acc=0.1657
Epoch 10: Train Loss=0.0974, Train Acc=0.1751, Val Loss=0.0977, Val Acc=0.1686
Epoch 11: Train Loss=0.0973, Train Acc=0.1759, Val Loss=0.0969, Val Acc=0.1738
Epoch 12: Train Loss=0.0963, Train Acc=0.1790, Val Loss=0.0964, Val Acc=0.1778
Epoch 13: Train Loss=0.0955, Train Acc=0.1837, Val Loss=0.095

In [172]:
stats2 = trainLabel(model, optimizer, criterion, train_dl, val_dl, 20)

Epoch 1: Train Loss=0.1785, Train Acc=0.0118, Val Loss=0.1585, Val Acc=0.0231
Epoch 2: Train Loss=0.1458, Train Acc=0.0356, Val Loss=0.1394, Val Acc=0.0503
Epoch 3: Train Loss=0.1335, Train Acc=0.0549, Val Loss=0.1306, Val Acc=0.0600
Epoch 4: Train Loss=0.1272, Train Acc=0.0669, Val Loss=0.1254, Val Acc=0.0736
Epoch 5: Train Loss=0.1239, Train Acc=0.0731, Val Loss=0.1229, Val Acc=0.0767


KeyboardInterrupt: 

In [187]:
stats3 = trainLabel(model, optimizer, criterion, train_dl, val_dl, 50)

Epoch 1: Train Loss=0.0860, Train Acc=0.2786, Val Loss=0.0567, Val Acc=0.4166
Epoch 2: Train Loss=0.0502, Train Acc=0.4644, Val Loss=0.0468, Val Acc=0.5017
Epoch 3: Train Loss=0.0445, Train Acc=0.5147, Val Loss=0.0440, Val Acc=0.5132
Epoch 4: Train Loss=0.0415, Train Acc=0.5431, Val Loss=0.0451, Val Acc=0.5101
Epoch 5: Train Loss=0.0393, Train Acc=0.5644, Val Loss=0.0399, Val Acc=0.5604
Epoch 6: Train Loss=0.0373, Train Acc=0.5825, Val Loss=0.0405, Val Acc=0.5537
Epoch 7: Train Loss=0.0359, Train Acc=0.5906, Val Loss=0.0373, Val Acc=0.5820
Epoch 8: Train Loss=0.0349, Train Acc=0.6046, Val Loss=0.0347, Val Acc=0.6080
Epoch 9: Train Loss=0.0335, Train Acc=0.6144, Val Loss=0.0366, Val Acc=0.5953
Epoch 10: Train Loss=0.0321, Train Acc=0.6339, Val Loss=0.0320, Val Acc=0.6397
Epoch 11: Train Loss=0.0308, Train Acc=0.6507, Val Loss=0.0319, Val Acc=0.6335
Epoch 12: Train Loss=0.0292, Train Acc=0.6625, Val Loss=0.0284, Val Acc=0.6711
Epoch 13: Train Loss=0.0286, Train Acc=0.6707, Val Loss=0.029

In [21]:
model2 = LabelToClusterModel()
model2.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
print(next(model2.parameters()).device)

cpu


In [22]:
stats4 = trainLabel(model2, optimizer, criterion, train_dl, val_dl, 50)

Epoch 1: Train Loss=0.0835, Train Acc=0.2914, Val Loss=0.0547, Val Acc=0.4224
Epoch 2: Train Loss=0.0481, Train Acc=0.4803, Val Loss=0.0441, Val Acc=0.5195
Epoch 3: Train Loss=0.0434, Train Acc=0.5228, Val Loss=0.0413, Val Acc=0.5399
Epoch 4: Train Loss=0.0407, Train Acc=0.5478, Val Loss=0.0430, Val Acc=0.5344
Epoch 5: Train Loss=0.0383, Train Acc=0.5706, Val Loss=0.0384, Val Acc=0.5766
Epoch 6: Train Loss=0.0372, Train Acc=0.5812, Val Loss=0.0364, Val Acc=0.5965
Epoch 7: Train Loss=0.0356, Train Acc=0.5969, Val Loss=0.0370, Val Acc=0.5817
Epoch 8: Train Loss=0.0345, Train Acc=0.6082, Val Loss=0.0386, Val Acc=0.5785
Epoch 9: Train Loss=0.0333, Train Acc=0.6206, Val Loss=0.0324, Val Acc=0.6318
Epoch 10: Train Loss=0.0321, Train Acc=0.6299, Val Loss=0.0351, Val Acc=0.6085
Epoch 11: Train Loss=0.0308, Train Acc=0.6441, Val Loss=0.0313, Val Acc=0.6425
Epoch 12: Train Loss=0.0295, Train Acc=0.6590, Val Loss=0.0336, Val Acc=0.6235
Epoch 13: Train Loss=0.0285, Train Acc=0.6717, Val Loss=0.029

In [23]:
torch.save(model2.state_dict(), "./labeltoclus.pt")

In [24]:
stats4 = trainLabel(model2, optimizer, criterion, train_dl, val_dl, 30)

Epoch 1: Train Loss=0.0176, Train Acc=0.7901, Val Loss=0.0196, Val Acc=0.7752
Epoch 2: Train Loss=0.0170, Train Acc=0.7958, Val Loss=0.0188, Val Acc=0.7883
Epoch 3: Train Loss=0.0165, Train Acc=0.7997, Val Loss=0.0198, Val Acc=0.7726
Epoch 4: Train Loss=0.0168, Train Acc=0.8021, Val Loss=0.0188, Val Acc=0.7829
Epoch 5: Train Loss=0.0167, Train Acc=0.7993, Val Loss=0.0222, Val Acc=0.7574
Epoch 6: Train Loss=0.0168, Train Acc=0.7994, Val Loss=0.0196, Val Acc=0.7722
Epoch 7: Train Loss=0.0165, Train Acc=0.8023, Val Loss=0.0170, Val Acc=0.8029
Epoch 8: Train Loss=0.0161, Train Acc=0.8079, Val Loss=0.0183, Val Acc=0.7894
Epoch 9: Train Loss=0.0164, Train Acc=0.8033, Val Loss=0.0224, Val Acc=0.7590
Epoch 10: Train Loss=0.0158, Train Acc=0.8091, Val Loss=0.0206, Val Acc=0.7654
Epoch 11: Train Loss=0.0161, Train Acc=0.8073, Val Loss=0.0206, Val Acc=0.7754
Epoch 12: Train Loss=0.0157, Train Acc=0.8134, Val Loss=0.0201, Val Acc=0.7764
Epoch 13: Train Loss=0.0159, Train Acc=0.8088, Val Loss=0.016

In [25]:
torch.save(model2.state_dict(), "./labeltoclus82.pt")

In [26]:
stats5 = trainLabel(model2, optimizer, criterion, train_dl, val_dl, 30)

Epoch 1: Train Loss=0.0147, Train Acc=0.8232, Val Loss=0.0143, Val Acc=0.8281
Epoch 2: Train Loss=0.0150, Train Acc=0.8186, Val Loss=0.0182, Val Acc=0.7911
Epoch 3: Train Loss=0.0144, Train Acc=0.8261, Val Loss=0.0184, Val Acc=0.7975
Epoch 4: Train Loss=0.0146, Train Acc=0.8255, Val Loss=0.0179, Val Acc=0.7937
Epoch 5: Train Loss=0.0141, Train Acc=0.8305, Val Loss=0.0182, Val Acc=0.7982
Epoch 6: Train Loss=0.0143, Train Acc=0.8270, Val Loss=0.0171, Val Acc=0.8073
Epoch 7: Train Loss=0.0151, Train Acc=0.8180, Val Loss=0.0177, Val Acc=0.8007
Epoch 8: Train Loss=0.0144, Train Acc=0.8261, Val Loss=0.0156, Val Acc=0.8207
Epoch 9: Train Loss=0.0142, Train Acc=0.8294, Val Loss=0.0175, Val Acc=0.8067
Epoch 10: Train Loss=0.0142, Train Acc=0.8295, Val Loss=0.0207, Val Acc=0.7763
Epoch 11: Train Loss=0.0141, Train Acc=0.8294, Val Loss=0.0172, Val Acc=0.8069
Epoch 12: Train Loss=0.0143, Train Acc=0.8282, Val Loss=0.0219, Val Acc=0.7719
Epoch 13: Train Loss=0.0146, Train Acc=0.8262, Val Loss=0.016

In [27]:
torch.save(model2.state_dict(), "./labeltoclus83.pt")

In [28]:
torch.save(model2, "./labeltocluster.pt")

In [31]:
model2.eval()
with torch.no_grad():
    result = model2(torch.Tensor([0.428, 0.254, 0.296, 0.574, 0.117, 0.0426, 0.139]))
    predictedCluster = result.argmax()
    print(predictedCluster)

tensor(147)


In [35]:
def trainLabelShow(model, optimizer, criterion, trainloader, testloader, num_epochs):
    model.train()
    stats = [[],[],[],[]]
#     print(device)
    for epoch in range(num_epochs):
        train_loss = 0.0
        train_acc = 0.0
        val_loss = 0.0
        val_acc = 0.0
        for data in trainloader:
            
            inputs, labels = data
#             inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.type(torch.LongTensor)
#             print(inputs)
#             print(labels)
            optimizer.zero_grad()

            outputs = model(inputs)
#             print(outputs)
            predicted_class = torch.argmax(outputs, dim=1)
#             print(predicted_class)
#             break
            loss = criterion(outputs, labels)
#             print(loss)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_acc += (predicted == labels).sum().item()
            

 # Validation loop
        with torch.no_grad():
            for inputs, labels in testloader:
#                 inputs, labels = inputs.to(device), labels.to(device)
                labels = labels.type(torch.LongTensor)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                print(outputs)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                print(predicted)
                val_acc += (predicted == labels).sum().item()
        
        # Print epoch statistics
        train_loss /= len(trainloader.dataset)
        stats[0].append(train_loss)
        train_acc /= len(trainloader.dataset)
        stats[1].append(train_acc)
        val_loss /= len(testloader.dataset)
        stats[2].append(val_loss)
        val_acc /= len(testloader.dataset)
        stats[3].append(val_acc)
        print(f'Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')

    print("Finished training")
    return stats

In [36]:
stats6 = trainLabelShow(model2, optimizer, criterion, train_dl, val_dl, 1)

tensor([[-132.5881, -107.1712,  -57.5664,  ...,  -86.6567,  -57.1106,
          -86.2756],
        [-201.3797, -103.2517,  -83.6761,  ..., -170.4473, -176.9719,
          -19.8537],
        [-117.1567, -117.2183,  -13.3918,  ...,  -72.0933,  -15.9328,
         -126.8125],
        ...,
        [-188.5918, -168.5737, -198.5928,  ..., -471.6318, -128.9237,
         -111.1760],
        [-102.5665,  -93.0625, -129.4049,  ..., -207.0862,  -71.1742,
         -132.1375],
        [ -36.2786, -217.9303,  -62.5150,  ...,  -90.3817,  -55.5528,
         -300.0201]])
tensor([ 79, 494, 278, 277, 138, 179, 213,  16, 286, 380, 110, 376, 114,   5,
        156, 350, 165, 382, 389, 489, 212, 133, 172, 295, 299, 164, 402, 472,
         26, 227,  68, 114])
tensor([[-143.5014,  -62.8365, -116.2002,  ..., -229.5602,  -40.6945,
         -189.1806],
        [ -96.2039,  -36.5788, -145.7584,  ..., -273.4749, -141.4435,
          -90.8523],
        [ -99.0718,  -97.4678,  -76.7373,  ..., -106.2732,  -90.1055,
   

tensor([[-104.3756,  -27.7793, -105.0586,  ..., -179.6802,  -94.3006,
         -262.0695],
        [-168.9839, -123.7714,  -22.5942,  ...,  -71.9496,  -73.0779,
         -200.9652],
        [ -75.9457, -130.5678, -122.7493,  ...,  -84.4755, -143.2252,
         -127.4490],
        ...,
        [-150.3860, -322.8056, -144.1790,  ..., -320.8495, -110.2605,
         -148.3498],
        [ -83.5510,  -95.6857, -126.9277,  ..., -289.9907, -119.3453,
          -88.1462],
        [-114.3871,  -90.3558,  -76.7928,  ...,  -48.4976, -156.1270,
         -357.3142]])
tensor([431, 466, 189, 480,  76, 422, 382, 247,  59, 180, 342, 369, 223, 203,
        101, 256,  35, 230,   3, 114, 349,  82, 274, 365, 306,  87, 452, 115,
        445,  92, 283, 177])
tensor([[  -4.5574,  -93.8817, -134.9250,  ...,  -84.1656, -202.5766,
         -328.4730],
        [-100.8776, -147.2899, -130.4227,  ...,  -77.3440, -171.0531,
         -252.1873],
        [-164.9270,  -45.1439, -166.5096,  ..., -331.2549, -125.2860,
   

tensor([[-112.9453, -121.5471,  -23.6403,  ..., -100.9020,  -60.9213,
          -23.1843],
        [ -96.5665,  -68.0593,  -72.9313,  ...,  -92.3043,  -87.8547,
         -265.8520],
        [-181.6787, -115.4577, -101.1301,  ..., -183.6039, -156.0027,
          -54.4040],
        ...,
        [ -55.6083, -101.3629, -181.6499,  ...,  -24.6632, -115.7679,
         -331.9672],
        [-120.2988, -127.3962,  -10.8468,  ...,  -76.6621, -108.0315,
         -113.2325],
        [-144.2991, -124.5458, -183.6825,  ...,  -75.9494, -187.7385,
         -250.6054]])
tensor([167, 103, 470, 152, 203, 376, 326,  51, 370, 344, 467, 486, 356, 307,
        283,  74, 192, 429, 281, 473, 457, 184, 167,  81,  51, 140, 266,   2,
        451, 311, 389, 444])
tensor([[ -80.3804,  -95.1336, -188.2400,  ...,  -66.6506,  -97.9699,
         -277.7505],
        [-194.4339, -148.8969,  -47.3927,  ..., -183.4388,  -11.6866,
         -107.8287],
        [-153.7027, -140.7023,  -71.5447,  ..., -160.8076,  -41.5472,
   

tensor([[ -88.5882,  -50.9213, -146.5505,  ..., -275.8574, -147.8972,
          -66.5146],
        [-105.4096, -114.9879, -150.6281,  ..., -361.2671, -163.7498,
         -125.6475],
        [-135.4119, -197.3588, -110.9110,  ..., -154.2842, -188.7342,
         -342.1379],
        ...,
        [ -75.5981, -122.1250, -120.7273,  ...,  -89.4200, -123.1901,
         -130.0427],
        [-101.8062, -117.5173,  -92.3557,  ..., -143.3186,  -63.6262,
          -88.9533],
        [ -35.0261, -125.8826, -236.5346,  ...,  -57.0532,  -74.4870,
         -230.3742]])
tensor([487, 306, 367, 348, 316,   5,   6, 138, 200, 108, 152, 307,  11, 434,
        450, 104, 465,  33,  27,  78,  20,  35, 189, 403, 332, 148, 413, 120,
        215,  31, 421, 305])
tensor([[ -67.2820, -189.9080, -103.8889,  ..., -213.1546,  -90.0691,
         -120.6468],
        [-139.3152, -157.2252,  -85.6562,  ..., -172.0491,  -58.7432,
         -227.8477],
        [-122.7861, -116.5274, -121.8335,  ..., -190.4023, -162.4790,
   

tensor([[ -66.9220, -135.0217, -183.8965,  ..., -103.9715, -134.2529,
         -132.2405],
        [-128.2411, -261.4336, -131.0006,  ..., -275.1654, -137.6234,
         -126.9548],
        [-140.3633,  -87.4941, -168.8906,  ...,  -51.9242, -127.5607,
         -169.0364],
        ...,
        [ -58.6093, -298.1146,  -67.1345,  ..., -138.0025,  -69.5297,
         -232.6283],
        [-228.3967, -155.5043,  -69.3619,  ..., -211.6247,  -23.5454,
          -70.0196],
        [-134.6711, -104.6754, -109.0731,  ..., -164.0387,  -56.3253,
         -427.4780]])
tensor([133, 118, 247,  44, 365, 270, 283,  70, 324,  76, 108, 246, 187, 279,
        240, 233, 264, 396, 422, 438, 170, 310,  54,  31, 307, 175,  33, 309,
         28, 262,  96, 238])
tensor([[-110.6042,  -47.6770, -180.2766,  ..., -360.3252, -134.9188,
          -86.1113],
        [-172.6753, -160.8311,  -87.6107,  ..., -179.8493,  -59.7378,
         -337.8511],
        [ -30.5777, -118.9152, -218.7767,  ...,  -57.9753,  -71.4041,
   

In [31]:
# Define transform to normalize data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download and load training data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Download and load test data
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [79]:
for i in trainloader:
    print(i)
    break

[tensor([[[[ 0.0902,  0.0588,  0.0588,  ...,  0.1059,  0.0824,  0.0588],
          [-0.0510, -0.1059, -0.1137,  ...,  0.0588,  0.0275,  0.0118],
          [-0.0824, -0.0275,  0.2314,  ...,  0.0196, -0.0118, -0.0353],
          ...,
          [-0.1608, -0.2157, -0.4353,  ..., -0.5216, -0.5922, -0.6000],
          [-0.2157, -0.2314, -0.4196,  ..., -0.4510, -0.4196, -0.4196],
          [-0.1529, -0.1529, -0.1843,  ..., -0.3804, -0.3725, -0.3882]],

         [[ 0.0431,  0.0196,  0.0196,  ...,  0.0588,  0.0353,  0.0118],
          [-0.1373, -0.1843, -0.2000,  ...,  0.0196, -0.0118, -0.0118],
          [-0.1843, -0.0824,  0.1529,  ..., -0.0196, -0.0431, -0.0431],
          ...,
          [-0.1529, -0.2314, -0.4431,  ..., -0.4353, -0.5059, -0.4902],
          [-0.2000, -0.2235, -0.4118,  ..., -0.3569, -0.3333, -0.3333],
          [-0.1451, -0.1373, -0.1765,  ..., -0.3176, -0.3020, -0.3176]],

         [[-0.6627, -0.7333, -0.7255,  ..., -0.0745, -0.0902, -0.0980],
          [-0.5137, -0.5373, 